
# KPI Evaluation Report

This notebook demonstrates how to compute key AI-linked KPIs from synthetic events data. The metrics include:

- **Ticket deflection %** – The share of customer intents resolved via self‑service or bots.
- **AI agent resolution %** – The percentage of bot-handled interactions resolved without human escalation.
- **Search success rate** – The proportion of searches that returned a useful result.
- **Onboarding time‑to‑value (TTV)** – The average number of days between user sign‑up and first successful task.
- **Article freshness** – The percentage of knowledge articles updated within the SLA window (e.g., 90 days).

We assume you have a `synthetic_events.csv` file in the `data` directory with event logs for searches, bot interactions, and onboarding milestones. The code below illustrates how to load the data and compute these metrics using pandas.


In [ ]:

import pandas as pd
from datetime import datetime

# Load synthetic events (replace with your actual CSV path)
synthetic_events = pd.read_csv('data/synthetic_events.csv')

# Example structure of synthetic_events.csv:
# event, timestamp, user_id, outcome
# search_success, 2025-10-10T12:00:00Z, U001, true
# bot_resolution, 2025-10-10T12:05:00Z, C001, true
# case_open, 2025-10-10T12:10:00Z, C002, 
# onboarding_start, 2025-10-01, U003, 
# onboarding_success, 2025-10-03, U003, 

# Ticket deflection % = (bot_resolution + self_service_resolution) / total_interactions
interactions = synthetic_events[synthetic_events['event'].isin(['bot_resolution','self_service_resolution','case_open'])]
total_intents = len(interactions)
deflected = len(interactions[interactions['event'].isin(['bot_resolution','self_service_resolution'])])
deflection_pct = (deflected / total_intents) * 100 if total_intents > 0 else 0

# AI agent resolution % = bot_resolution / bot_interactions
bot_interactions = len(interactions[interactions['event'].isin(['bot_interaction','bot_resolution'])])
bot_resolved = len(interactions[interactions['event'] == 'bot_resolution'])
ai_resolution_pct = (bot_resolved / bot_interactions) * 100 if bot_interactions > 0 else 0

# Search success rate = search_success / total_searches
searches = synthetic_events[synthetic_events['event'].isin(['search_success','search_failure'])]
total_searches = len(searches)
search_success = len(searches[searches['event'] == 'search_success'])
search_success_pct = (search_success / total_searches) * 100 if total_searches > 0 else 0

# Onboarding time-to-value (days)
onboarding = synthetic_events[synthetic_events['event'].isin(['onboarding_start','onboarding_success'])]
ttv_records = onboarding.pivot_table(index='user_id', columns='event', values='timestamp', aggfunc='first')
# Convert timestamps to datetime
for col in ['onboarding_start','onboarding_success']:
    ttv_records[col] = pd.to_datetime(ttv_records[col], errors='coerce')

ttv_records['ttv_days'] = (ttv_records['onboarding_success'] - ttv_records['onboarding_start']).dt.days
avg_ttv_days = ttv_records['ttv_days'].dropna().mean()

# Article freshness requires knowledge_articles data (not shown here)

print(f"Ticket deflection %: {deflection_pct:.2f}%")
print(f"AI agent resolution %: {ai_resolution_pct:.2f}%")
print(f"Search success rate: {search_success_pct:.2f}%")
print(f"Average onboarding time-to-value (days): {avg_ttv_days:.2f}")



## Results and Next Steps

The metrics above give a snapshot of your system performance. To visualize trends, consider plotting these metrics over time or by segment. Use `matplotlib` or `pandas` plotting functions to generate bar or line charts and save them into the `docs/metrics/` directory for inclusion in reports or dashboards.

Further improvements:
- Integrate real event logs instead of synthetic data.
- Expand the schema to include error types and response times.
- Automate the notebook run in CI to generate fresh reports on demand.
